<a href="https://colab.research.google.com/github/paolopoemape/Lab1/blob/master/Copy_of_Example_Walk_through_for_ER_Hero.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

####  Background and description of data    ######
There were two separate user groups who simulated being in an Emergency Room (ER) waiting area. 

Group 1 used an educational game called "ER Hero" to learn about the functions and operations in the ER. This group is labeled "1" in column 2 of the data.

Group 2 read an electronic informational document that had the same information. This group is labeled "0" in column 2 of the data.

Both groups took a survey both before AND after they simulated waiting in the ER. Survey questions used a 1-7 scale, with 1= strongly disagree and 7=strongly agree.

The columns in the data set are labeled "Pre" for survey responses that occurred prior to the exercise, and "post" for survey responses that occurred after the exercise.

Functional Recall, Interest, Enjoyment, and Nervousness were measured. Let's try and answer the first basic question: 

**Did the game generate more interest in ER operations, compared to the informational document?**

These were the survey questions that were asked that were intended to measure interest: 

PreInterest1: I expect to feel bored.   
PreInterest2: I expect to feel stimulated.   
PreInterest3: I expect to feel interested.   
PreInterest4: I expect to feel indifferent.   

PostInterest1: I felt bored.   
PostInterest2: I felt stimulated.   
PostInterest3: I felt interested.   
PostInterest4: I felt indifferent.  
 

In [18]:
#load libraries
!pip install pingouin
import pingouin as pg
import pandas as pd
import statsmodels.api as sm

In [25]:
#import the data
ABtestdata = pd.read_csv('https://byu.box.com/shared/static/d5czoanqw52fujp1p193iz53tb0k5g9r')

#remove cases with missing values
ABtestdata = ABtestdata.dropna()

#preview what the data looks like
ABtestdata.head()

#(optionally) view all of the column names (uncomment if desired)
# list(ABtestdata.columns)

,UserID,Gamer,EstTime,SRPrefunct,SRPreprof,SRPostfunct,SRPostprof,ChangeSRFunct,ChangeSRProf,prefunct1,prefunct2,prefunct3,prefunct4,prefunct5,prefunct6,PreFunctCorrect,preprof1,preprof2,preprof3,preprof4,preprof5,preprof6,PreProfCorr,TotRightPre,postfunct1,postfunct2,postfunct3,postfunct4,postfunct5,postfunct6,PostFunctCorr,postprof1,postprof2,postprof3,postprof4,postprof5,postprof6,PostProfCorr,TotalRightPost,ChangeTotScore,...,InterConfPre2,InterConfPre3,InterConfPre4,L2DPre1,L2DPre2,L2DPre3,L2DPre4,InterConfPost1,InterConfPost2,InterConfPost3,InterConfPost4,L2DPost1,L2DPost2,L2DPost3,L2DPost4,PostEoU1,PostEoU2,PostEoU3,PostEoU4,PostEoU5,PostEoU6,PostEoU7,PostEoU8,PostEoU9,UseWaitingRoom,Male,Age,Marital,InterestPre,InterestPost,NervousPre,NervousPost,JoyPre,JoyPost,FrustPre,InteractConfPre,InteractConfPost,LikelihoodDiscPre,LikelihoodDiscPost,PEOU
0,45,1,8,1,1,3,3,2,2,1,10,4,10,10,3,3,10,10,4,10,1,10,4,7,10,10,10,10,10,10,6,10,10,10,10,10,10,6.0,12,5,...,2.0,2.0,3.0,6.0,3.0,6.0,5.0,5,4,5,4,6,5,5,5,6,6,6,6,3,5,5,6,6,7,1,22,1,2.25,2.75,1.2,2.2,3.0,3.4,2.5,2.25,4.50,5.000000,5.333333,5.444444
1,95,1,12,2,2,4,4,2,2,1,1,2,10,10,10,3,10,10,4,10,2,10,4,7,10,10,4,10,10,10,5,10,10,4,10,10,10,5.0,10,3,...,5.0,5.0,5.0,6.0,5.0,5.0,5.0,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,1,22,1,4.25,5.50,2.0,2.2,5.8,4.2,2.0,5.00,6.00,5.333333,6.000000,6.000000
2,41,1,15,2,1,4,4,2,3,10,10,4,10,10,10,5,10,10,10,10,2,10,5,10,10,10,4,10,10,10,5,10,10,10,10,10,10,6.0,11,1,...,5.0,6.0,6.0,6.0,6.0,6.0,6.0,6,5,6,6,6,6,6,6,7,7,7,6,6,6,6,6,7,7,0,26,1,2.75,6.25,2.8,1.8,4.0,4.6,2.0,4.75,5.75,6.000000,6.000000,6.444444
3,38,1,20,4,4,5,5,1,1,10,10,10,10,2,10,5,10,10,10,10,10,10,6,11,10,10,10,10,2,10,5,10,10,10,10,10,10,6.0,11,0,...,5.0,5.0,6.0,6.0,6.0,6.0,6.0,6,6,6,6,6,6,6,6,6,6,6,6,2,4,6,6,6,6,1,23,1,5.25,3.75,2.0,2.8,5.0,3.8,2.0,5.25,6.00,6.000000,6.000000,5.333333
4,27,1,10,2,2,4,3,2,1,1,3,4,10,2,10,2,10,10,4,4,2,10,3,5,10,3,4,10,10,10,4,10,10,10,10,2,10,5.0,9,4,...,3.0,4.0,5.0,5.0,5.0,5.0,6.0,5,3,4,5,5,6,6,6,6,6,5,5,5,4,6,5,6,7,0,21,1,3.50,5.00,2.0,2.0,5.4,4.0,2.0,3.75,4.25,5.000000,5.666667,5.333333


In [26]:
##The patient's interest level was one of the outcomes being measured. 
##Let's find out if the educational game affected interest different than the educational document

#Interest was measured in the survey both before and after the experience. Each survey asked five questions about interest.
#Some questions were worded reverse from others (e.g., "I did not find this interesting" vs. "I found this interesting"). The scoring for these questions needs to be reversed.
ABtestdata['PreNervous1'] = 8- ABtestdata['PreNervous1']
ABtestdata['PreNervous4'] = 8- ABtestdata['PreNervous4']
ABtestdata['PostNervous1'] = 8- ABtestdata['PostNervous1']
ABtestdata['PostNervous4'] = 8- ABtestdata['PostNervous4']
print ('done!')

done!


In [42]:
#Cronbach's alpha is a measure that estimates how well our questions match one another (i.e., whether they all measure the same thing).
#As a rule of thumb, a Cronbach's alpha of .7 or better is considered good. If it is lower than .7, we look at dropping one or more questions that may not be a good match to the others
preinterestDF = ABtestdata[['PreNervous1','PreNervous2','PreNervous5']]
postinterestDF = ABtestdata[['PostNervous1','PostNervous2', 'PostNervous5']]
print("Interest Pre-nervous: ", pg.cronbach_alpha(data=preinterestDF))
print("Interest Post-nervous: ", pg.cronbach_alpha(data=postinterestDF))

Interest Pre-nervous:  (0.757260537550358, array([0.608, 0.856]))
Interest Post-nervous:  (0.8493802094464629, array([0.757, 0.911]))


In [22]:
#since it looks like our measures of interest are okay, let's create a column that is an average of all interest questions together, for each survey
ABtestdata['NervousPre'] = preinterestDF.mean(axis=1)
ABtestdata['NervousPost'] = postinterestDF.mean(axis=1)
(ABtestdata[['UserID','NervousPre','NervousPost']]).head()

,UserID,NervousPre,NervousPost
0,45,4.4,4.2
1,95,4.4,4.6
2,41,3.2,4.6
3,38,4.8,4.0
4,27,4.4,4.4


In [23]:
#find out the average interest level of each group BEFORE they started and AFTER they finished 
ABtestdata.groupby('Gamer')[['NervousPre','NervousPost']].mean()

,NervousPre,NervousPost
Gamer,,
0,4.217391,4.226087
1,4.368000,4.392000


In [24]:
regmodel = sm.OLS(ABtestdata['NervousPost'],ABtestdata[['Gamer','NervousPre','Male','Age']]).fit()
output = regmodel.params.to_frame(name="effect")
output['significance'] = regmodel.pvalues
output.round(decimals=2)

,effect,significance
Gamer,0.10,0.47
NervousPre,0.87,0.00
Male,-0.00,0.99
Age,0.02,0.03


You should notice that interest level before the experience unsurprisingly impacts their interest level afterward. The game also provided a more interesting experience.
Please answer the following questions:
1. About how much interest did the ER Hero game generate, compared to the informational document?
2. What impact did age have on interest?
3. How much more interested were female vs. male participants?
4. What other questions do you think the survey should have asked?

5. Did the game provide a less nervous experience compared to the informational document? If so, about how much?

 PreNervous1: I expect to be at ease.  
 PreNervous2: I expect to be comfortable.   
 PreNervous3: I expect to be nervous.  
 PreNervous4: I expect to be upset.  
 PreNervous5: I expect to be calm.  

 PostNervous1: I was at ease.  
 PostNervous2: I was comfortable.  
 PostNervous3: I was nervous.  
 PostNervous4: I was upset.  
 PostNervous5: I was calm.  